
# RSVP Entropy–Fields Demo (Synthetic)

This notebook provides a **first-pass synthetic demonstration** of RSVP-style analyses described in the manuscript:

- A small 2D lattice that evolves scalar capacity Φ, entropy S, and a vector field **v**.
- A simple estimator for entropy production rate \(\dot{\Sigma}\).
- A sweep of the coupling parameter \(\lambda\) to generate a coarse phase map in the \(\lambda\)–\(\langle \dot{\Sigma} \rangle\) plane.
- A short time-evolution simulation to illustrate basic metrics (coherence proxy, Wasserstein-1 via 1D projection, etc.).

> Replace the synthetic update rules with your true lattice/spectral solvers when ready.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

# Deterministic seed for reproducibility
rng = np.random.default_rng(42)


In [ ]:

@dataclass
class LatticeConfig:
    nx: int = 64
    ny: int = 64
    dt: float = 0.05
    steps: int = 400
    diffusion_phi: float = 0.2
    diffusion_s: float = 0.15
    advection_scale: float = 0.4
    noise_scale: float = 0.02

def init_fields(cfg: LatticeConfig):
    # Scalar Φ: capacity; start with smooth gradient + noise
    X, Y = np.meshgrid(np.linspace(-1,1,cfg.nx), np.linspace(-1,1,cfg.ny), indexing='ij')
    Phi = 1.0 + 0.5*np.exp(-3*(X**2 + Y**2)) + 0.05*rng.standard_normal((cfg.nx, cfg.ny))
    # Entropy S: random-ish baseline
    S = 0.8 + 0.1*rng.standard_normal((cfg.nx, cfg.ny))
    # Vector field v = (vx, vy): initialized to small curls
    vx = 0.2*(-Y) + 0.02*rng.standard_normal((cfg.nx, cfg.ny))
    vy = 0.2*(X)  + 0.02*rng.standard_normal((cfg.nx, cfg.ny))
    return X, Y, Phi, S, vx, vy


In [ ]:

def roll2(a, dx, dy):
    return np.roll(np.roll(a, dx, axis=0), dy, axis=1)

def grad(a):
    # central differences with periodic boundaries
    ax = 0.5*(roll2(a, +1, 0) - roll2(a, -1, 0))
    ay = 0.5*(roll2(a, 0, +1) - roll2(a, 0, -1))
    return ax, ay

def laplacian(a):
    return (roll2(a,1,0)+roll2(a,-1,0)+roll2(a,0,1)+roll2(a,0,-1)-4*a)

def divergence(ax, ay):
    dax = 0.5*(roll2(ax, +1, 0) - roll2(ax, -1, 0))
    day = 0.5*(roll2(ay, 0, +1) - roll2(ay, 0, -1))
    return dax + day


In [ ]:

def step(Phi, S, vx, vy, cfg: LatticeConfig, lam: float):
    # Compute R = Phi - lam * S
    R = Phi - lam*S

    # Velocity update: v_t = -∇R + small curl relaxation via Laplacian
    dRx, dRy = grad(R)
    vx = vx - cfg.dt * dRx + 0.05 * laplacian(vx)
    vy = vy - cfg.dt * dRy + 0.05 * laplacian(vy)

    # Advection of Phi and S by v, plus diffusion
    adv_phi = divergence(Phi*vx, Phi*vy)
    adv_s   = divergence(S*vx,   S*vy)

    Phi = Phi - cfg.dt * adv_phi + cfg.diffusion_phi * laplacian(Phi) + cfg.noise_scale*rng.standard_normal(Phi.shape)
    S   = S   - cfg.dt * adv_s   + cfg.diffusion_s   * laplacian(S)   + cfg.noise_scale*rng.standard_normal(S.shape)

    # Entropy production estimator: Σ̇ ≈ ⟨∇S · v⟩ (lattice average)
    dSx, dSy = grad(S)
    sigma_dot = np.mean(dSx*vx + dSy*vy)

    # Coherence proxy: negative gradient energy of R (smaller => smoother)
    dRx, dRy = grad(R)
    coherence = -np.mean(dRx**2 + dRy**2)

    return Phi, S, vx, vy, sigma_dot, coherence


In [ ]:

cfg = LatticeConfig(nx=64, ny=64, dt=0.05, steps=150)
lambdas = np.linspace(0.0, 1.0, 21)

avg_sigma = []
avg_coherence = []

for lam in lambdas:
    X, Y, Phi, S, vx, vy = init_fields(cfg)
    sigmas = []
    coherences = []
    for _ in range(cfg.steps):
        Phi, S, vx, vy, sdot, coh = step(Phi, S, vx, vy, cfg, lam)
        sigmas.append(sdot)
        coherences.append(coh)
    avg_sigma.append(np.mean(sigmas[-50:]))       # late-time average
    avg_coherence.append(np.mean(coherences[-50:]))
    
# Plot λ vs <Σ̇> and λ vs coherence proxy
plt.figure(figsize=(6,4))
plt.plot(lambdas, avg_sigma, marker='o')
plt.xlabel('lambda')
plt.ylabel('late-time <Sigma_dot>')
plt.title('Phase Trend: lambda vs <Sigma_dot>')
plt.show()

plt.figure(figsize=(6,4))
plt.plot(lambdas, avg_coherence, marker='o')
plt.xlabel('lambda')
plt.ylabel('late-time coherence proxy')
plt.title('Phase Trend: lambda vs coherence')
plt.show()


In [ ]:

lam_demo = 0.4
cfg_demo = LatticeConfig(nx=64, ny=64, dt=0.05, steps=200)
X, Y, Phi, S, vx, vy = init_fields(cfg_demo)

sigma_hist = []
coh_hist = []

snapshots = []
snap_times = [20, 60, 120, 200]

for t in range(1, cfg_demo.steps+1):
    Phi, S, vx, vy, sdot, coh = step(Phi, S, vx, vy, cfg_demo, lam_demo)
    sigma_hist.append(sdot)
    coh_hist.append(coh)
    if t in snap_times:
        snapshots.append((t, Phi.copy(), S.copy()))

# Plot time series
plt.figure(figsize=(6,4))
plt.plot(sigma_hist)
plt.xlabel('time step')
plt.ylabel('Sigma_dot')
plt.title('Time evolution of Sigma_dot (lambda=0.4)')
plt.show()

plt.figure(figsize=(6,4))
plt.plot(coh_hist)
plt.xlabel('time step')
plt.ylabel('coherence proxy')
plt.title('Time evolution of coherence proxy (lambda=0.4)')
plt.show()

# Show a few scalar field snapshots (Φ)
for t, Phi_snap, S_snap in snapshots:
    plt.figure(figsize=(5,4))
    plt.imshow(Phi_snap, origin='lower', interpolation='nearest')
    plt.title(f'Phi snapshot at t={t} (lambda=0.4)')
    plt.colorbar()
    plt.show()

# Show corresponding S snapshots
for t, Phi_snap, S_snap in snapshots:
    plt.figure(figsize=(5,4))
    plt.imshow(S_snap, origin='lower', interpolation='nearest')
    plt.title(f'S snapshot at t={t} (lambda=0.4)')
    plt.colorbar()
    plt.show()


In [ ]:

# Wasserstein-1 proxy using 1D projection: compare row means across time
def w1_proxy(series_a, series_b):
    # Equal-mass discrete distributions on a line; here just L1 distance between CDFs
    a = np.cumsum(series_a / np.sum(series_a))
    b = np.cumsum(series_b / np.sum(series_b))
    return np.mean(np.abs(a - b))

# Compute proxy over snapshots of S (row-averaged)
proj_series = [np.mean(snap[2], axis=1) for snap in snapshots]  # S snapshots row mean
w1_vals = []
for i in range(len(proj_series)-1):
    w1_vals.append(w1_proxy(proj_series[i], proj_series[i+1]))

plt.figure(figsize=(6,4))
plt.plot(w1_vals, marker='o')
plt.xlabel('snapshot interval')
plt.ylabel('W1 proxy')
plt.title('W1 proxy across S snapshots (lambda=0.4)')
plt.show()



## Notes for Extension

- Replace `step(...)` with calls into your **real** solvers:
  - `simulation/lattice_solver.py` for finite-difference updates.
  - `simulation/spectral_solver.py` for Fourier-domain updates.
  - `simulation/stochastic_dynamics.py` for Langevin/MCMC noise models.
- Swap the synthetic \(\dot{\Sigma}\) estimator with your formal definition.
- Log results to JSONL and integrate with `utils/logging_utils.py` and `analysis/meta_analysis.py`.
- Use `experiments/run_entropy_stress.py` to batch λ and lattice size for robust phase boundary estimates.
